In [1]:
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import torch
from torchvision import transforms

from torch import nn

from PIL import Image

import os, random

In [2]:
# model load
CLASS_NUM = 4
model = torch.load('model/epoch_4_val_0.94.pt').to('cpu')
print(model)

# cf_fc = nn.Linear(1000, CLASS_NUM)
# model.add_module('CF_fc', cf_fc)
# stm = nn.Softmax(dim=-1)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# classes => 꽃개오동:0 참취:1 곰취:2 개오동:3
classes = ['Kkochgaeodong_Y', 'Chamchwi_Y', 'Gomchwi_N', 'Gaeodong_N']

# 이미지 transform
# topilimage = transforms.ToPILImage()
# totensor = transforms.ToTensor()
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# Web
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def image_predict():
    if request.method == 'POST':
        f = request.files['file']
        file_name = secure_filename(f.filename)
        img = Image.open(f)
        
        img.save('./static/image/uploads/' + file_name)
        img = Image.open('./static/image/uploads/' + file_name)
        img = transform(img)
        # img = img / 255.0

        # 이미지의 클래스 예측
        imgs = img.unsqueeze(0) # batch index 넣어줌
        predicted = model(imgs)
        predicted = model.CF_fc1(predicted)
        predicted = model.CF_fc2(predicted)
        print(predicted)
        # predicted = stm(predicted)
        predicted_class_num = torch.argmax(predicted)
        # 확인용 터미널 출력
        print(f"예측 결과는? {predicted}")
        print(f"예측은 class 번호는 ? {predicted_class_num}")
        pred_pro = f"%0.4f" % predicted[0][predicted_class_num]

        # 예시 이미지 선택
        ex_num = 3
        pred_name = classes[predicted_class_num]
        ex_list = os.listdir('static/image/examples/' + pred_name)
        ex_idx = random.sample(range(len(ex_list)), ex_num) # 3개 예시 뽑음
        
        ex_file_name = []
        for x in ex_idx:
            ex_file_name.append(ex_list[x])
        
    return render_template('result.html', 
    upload_image='image/uploads/' + file_name, 
    example_image=ex_file_name, #'image/examples/' + pred_name + '/' + ex_file_name,
    result_pro = pred_pro,
    result_name = pred_name,
    file_name = f.filename)

if __name__ == '__main__':
    app.run(port=2008, debug=True, use_reloader=False)
    #app.run(port=2033, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:2008
Press CTRL+C to quit
127.0.0.1 - - [30/May/2023 23:11:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:11:14] "GET /static/image/index_ex/887373_ori.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:11:14] "GET /static/image/index_ex/꽃개오동_잎_1018234.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:11:14] "GET /static/image/index_ex/꽃개오동_잎_1056952.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:12:18] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:12:18] "GET /static/image/uploads/887431.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:12:18] "GET /static/image/examples/Kkochgaeodong_Y/꽃개오동_잎_1056946.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:12:18] "GET /static/image/examples/Kkochgaeodong_Y/꽃개오동_잎_1121538.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2023 23:12:18] "GET /static/image/examples/Kkochgaeodong_Y/꽃개오동_잎_1018227.jpg HTTP/1.1" 200 -


tensor([[ 3.2158, -3.4052, -1.9436,  2.8504]], grad_fn=<AddmmBackward0>)
예측 결과는? tensor([[ 3.2158, -3.4052, -1.9436,  2.8504]], grad_fn=<AddmmBackward0>)
예측은 class 번호는 ? 0
